In [47]:
url_1 = "https://odre.opendatasoft.com/explore/embed/dataset/conso-departement-annuelle/table/?disjunctive.libelle_departement&disjunctive.libelle_region&disjunctive.e_operateurs&disjunctive.g_operateurs&refine.annee=2021&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQVZHIiwieUF4aXMiOiJjb25zb3RvdGFsZSIsInNjaWVudGlmaWNEaXNwbGF5Ijp0cnVlLCJjb2xvciI6IiM2NmMyYTUifV0sInhBeGlzIjoibGliZWxsZV9kZXBhcnRlbWVudCIsIm1heHBvaW50cyI6NTAsInNvcnQiOiIiLCJjb25maWciOnsiZGF0YXNldCI6ImNvbnNvLWRlcGFydGVtZW50LWFubnVlbGxlIiwib3B0aW9ucyI6eyJkaXNqdW5jdGl2ZS5saWJlbGxlX2RlcGFydGVtZW50Ijp0cnVlLCJkaXNqdW5jdGl2ZS5saWJlbGxlX3JlZ2lvbiI6dHJ1ZSwiZGlzanVuY3RpdmUuZV9vcGVyYXRldXJzIjp0cnVlLCJkaXNqdW5jdGl2ZS5nX29wZXJhdGV1cnMiOnRydWUsInJlZmluZS5hbm5lZSI6IjIwMjEifX19XSwidGltZXNjYWxlIjoiIiwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D&location=3,17.56025,53.4375&basemap=jawg.light"
url_2 = "https://www.insee.fr/fr/statistiques/6436484?sommaire=6036904#tableau-figure1_radio1"
url_3 = "https://odre.opendatasoft.com/explore/dataset/temperature-quotidienne-departementale/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=false"
url_4 = "https://www.insee.fr/fr/statistiques/6436484?sommaire=6036904#tableau-figure1_radio1"
url_5 = "https://www.observatoire-des-territoires.gouv.fr/outils/cartographie-interactive/#c=indicator&i=insee_rp_hist_1968.part_logt_vacant&s=2020&view=map9"
url_6 = "https://ufe-electricite.fr/watt-the-carte/deploiement-bornes-de-recharge-en-france/dans-les-territoires/"
url_7 = "https://www.carburants.org/borne-electrique/departements/"

In [263]:
# Installation des packages

!pip install -q lxml
!pip install webdriver-manager
!pip install BeautifulSoup4

import requests
import re
import bs4
import lxml
import pandas as pd
import urllib

In [54]:
# Webscrapping

In [259]:
# Nous avons besoin de scraper la donnée "population" sur chaque page Wikipédia des départements, et pour cela, nous devons obtenir les URL de chaque page. 
# Ainsi, nous créons une liste qui recense tous les URL nécessaires pour accéder à chaque page Wikipédia et extraire la donnée de population.

url_dep = "https://fr.wikipedia.org/wiki/D%C3%A9partement_fran%C3%A7ais"
text_dep = request.urlopen(url_dep).read()
page_dep = bs4.BeautifulSoup(text_dep, 'lxml')
tableau_dep = page_dep.find('div', {'class':'colonnes'})
lignes_dep = tableau_dep.find_all('li')

list_url=[]
for i in range(len(lignes_dep)):
    if re.search(r'href="([^"]*)"', str(lignes_dep[i])).group(1) not in ["#cite_note-collectivitésOM-178",'/wiki/Guadeloupe','/wiki/La_R%C3%A9union'] :
        list_url.append("https://fr.wikipedia.org"+re.search(r'href="([^"]*)"', str(lignes_dep[i])).group(1))

In [261]:
# Nous automatisons le webscraping d'une page Wikipédia, et chaque résultat est stocké dans un dictionnaire.

dico_dep = dict()
k=1
for url in list_url:
    text = request.urlopen(url).read()
    page = bs4.BeautifulSoup(text, 'lxml')
    tableau = page.find('table',{'class':'infobox_v2 noarchive'})
    tableau_body = tableau.find('tbody')
    lignes = tableau_body.find_all('tr')
    nom_dep = lignes[0].find('td').text.strip()
    if 1 <= k <= 19:
        num_dep = str(k)
    elif k == 20:
        num_dep = '2A'
    elif k == 21:
        num_dep = '2B'
    else:
        num_dep = str(k - 1)
    k=k+1
    for ligne in lignes:
        cols1 = ligne.find_all('th')
        cols2 = ligne.find_all('td')
        cols = [ele.text.strip() for ele in cols1]+[elem.text.strip() for elem in cols2]
        if cols[0]=='Population' or cols[0]=='Populationmunicipale':
            population = re.sub(r'hab.*', '', cols[1])
    dico_dep[num_dep] = {'nom_dep': nom_dep, 'population': population}


In [264]:
# On transforme le dictionnaire en base de données

data_dep = pandas.DataFrame.from_dict(dico_dep,orient='index')

In [266]:
# On visualise la base de donnée

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
data_dep.head()

,nom_dep,population
1,Ain,657 856
2,Aisne,529 374
3,Allier,335 628
4,Alpes-de-Haute-Provence,165 451
5,Hautes-Alpes,140 605


In [249]:
# Test pour récupérer la population sur une page wikipédia

# On récupère l'url de la page wikipédia et on importe le texte HTML de la page

url_Loire_Atlantique = "https://fr.wikipedia.org/wiki/Loire-Atlantique"
text_Loire_Atlantique = request.urlopen(url_Loire_Atlantique).read()

In [250]:
# On recherche de manière efficace les balises de la page
population_Loire_Atlantique = 0
page_Loire_Atlantique = bs4.BeautifulSoup(text_Loire_Atlantique, 'lxml')

# On récupère le tableau souhaité
tableau_Loire_Atlantique = page_Loire_Atlantique.find('table',{'class':'infobox_v2 noarchive'})

# On extrait le corps du tableau
tableau_Loire_Atlantique_body = tableau_Loire_Atlantique.find('tbody')

# On extrait toutes les lignes
lignes = tableau_Loire_Atlantique_body.find_all('tr')


#On affiche uniquement le texte et on supprime le code (avec la fonction .text.strip) et on extrait la ligne population du tableau
for ligne in lignes:
    cols1 = ligne.find_all('th')
    cols2 = ligne.find_all('td')
    cols = [ele.text.strip() for ele in cols1]+[elem.text.strip() for elem in cols2]
    if cols[0]=='Population':
        population_Loire_Atlantique = cols[1]

print(population_Loire_Atlantique)

1 445 171 hab. (2020)


In [55]:
# Webscrapping des logements vacants

text_logements_vacants = request.urlopen(url_5).read()
page_logements_vacants = bs4.BeautifulSoup(text_logements_vacants, 'lxml')
tableau_logements_vacants = page_logements_vacants.find('table', {'id': 'tm_datatable'})
print(tableau_logements_vacants)
## A partir d'ici, pas sûr d'avoir récuperer la bonne partie du code HTML qui code le tableau



None


In [51]:
print(page_logements_vacants)

In [53]:
text_logements_vacants = request.urlopen(url_5).read()
page_logements_vacants = bs4.BeautifulSoup(text_logements_vacants, 'lxml')
print(page_logements_vacants)